In [9]:
import csv
import sys
import time
import work

from tqdm import tqdm

In [8]:
path = f'./tsv/front-512/42'

st_time = time.time()
BUFFER_SIZE = int(8192*640/8)

csv.field_size_limit(sys.maxsize)

train = work.load_tsv_dataset(f'{path}/train.tsv')
valid = work.load_tsv_dataset(f'{path}/dev.tsv')
test = work.load_tsv_dataset(f'{path}/test.tsv')

print(len(train))
print(len(valid))
print(len(test))

2963
370
371


In [3]:
#2,621,440文字が限界#
wakati = MeCab.Tagger("-Owakati -b 5242880")

for work in tqdm(train):
    content = work['content']
    work['wakati'] = ""
    if len(content) > BUFFER_SIZE:
        content = content[:BUFFER_SIZE]
    parsed = wakati.parse(content)
    try:
        tokens = parsed.split()
    except AttributeError as err:
        print("error:", err)
        print("length:", len(content))
        print("content:", parsed)
    work['wakati'] += " ".join(tokens) + " "

for work in tqdm(valid):
    content = work['content']
    work['wakati'] = ""
    if len(content) > BUFFER_SIZE:
        content = content[:BUFFER_SIZE]
    parsed = wakati.parse(content)
    try:
        tokens = parsed.split()
    except AttributeError as err:
        print("error:", err)
        print("length:", len(content))
        print("content:", parsed)
    work['wakati'] += " ".join(tokens) + " "
    
for work in tqdm(test):
    content = work['content']
    work['wakati'] = ""
    if len(content) > BUFFER_SIZE:
        content = content[:BUFFER_SIZE]
    parsed = wakati.parse(content)
    try:
        tokens = parsed.split()
    except AttributeError as err:
        print("error:", err)
        print("length:", len(content))
        print("content:", parsed)
    work['wakati'] += " ".join(tokens) + " "
    
x_train = [work['wakati'] for work in train]
x_valid = [work['wakati'] for work in valid]
x_test  = [work['wakati'] for work in test ]

y_train = [work['label'] for work in train]  
y_valid = [work['label'] for work in valid]
y_test  = [work['label'] for work in test ]

del(train)
del(valid)
del(test)

print(len(x_train), len(y_train))
print(len(x_valid), len(y_valid))
print(len(x_test), len(y_test))

100%|█████████████████████████████████████████████████████████████████████████████████| 371/371 [00:51<00:00,  7.15it/s]

2963 2963
370 370
371 371


In [4]:
classifier = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', OneVsRestClassifier(LogisticRegression()))
])
classifier.fit(x_train, y_train)
print(time.strftime("%H:%M:%S", time.gmtime(time.time()-st_time)))

00:10:25


In [5]:
predicted = classifier.predict(x_valid)
#valid_labels = mlb.inverse_transform(predicted)
print(time.strftime("%H:%M:%S", time.gmtime(time.time()-st_time)))
print(accuracy_score(y_valid, predicted))

00:10:32
0.7648648648648648


In [6]:
st_time = time.time()
predicted = classifier.predict(x_test)
#valid_labels = mlb.inverse_transform(predicted)
print(time.strftime("%H:%M:%S", time.gmtime(time.time()-st_time)))
print(accuracy_score(y_test, predicted))

00:00:07
0.7547169811320755


In [34]:
### SAVE MODEL ###
import pickle

# save
with open('front512_LR.pkl','wb') as f:
    pickle.dump(classifier, f)